## Misc. Data Analysis

This file contains various smaller chunks of code used for figures and analysis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter, find_peaks

In [ ]:
# The first time data from the scope is loaded, the first 21 lines of header need to be chopped off

# for i in range(0, 57):
#     file_path = f"tek{i:04}ALL.csv"
#     with open(file_path, "r"
#               ) as file:
#         lines = file.readlines()[21:]  # Skip the first 21 lines
#     with open(file_path, "w") as file:
#         file.writelines(lines)

## Hyperfine Splitting

In [ ]:
## Load Hyperfine Data

data = []
scope = []
arr = np.loadtxt(f"tek0000ALL.csv",
                 delimiter=",", dtype=str)

data.append(arr[2:, 0].astype(float)) # Append common time
scope.append(arr[2:, 0].astype(float))
for i in range(0, 50):
    arr = np.loadtxt(f"tek{i:04}ALL.csv",
                 delimiter=",", dtype=str)
    data.append(arr[2:, 1].astype(float))
    scope.append(arr[2:, 2].astype(float))
data = np.array(data).T
scope = np.array(scope).T


In [ ]:
## Large Hyperfine Spacing Figure

def filter_by_limit(X, Y, min_val, max_val):
    X1 = [i for i in X if min_val <= i <= max_val]
    Y1 = [Y[i] for i, x in enumerate(X) if min_val <= x <= max_val]
    return np.array(X1), np.array(Y1)

L = .539  # Cavity Length
c = 299792458
FC = c / (2 * L) * 1e-6

x, y = filter_by_limit(data[:, 0], data[:, 1], -0.053, -.003)
sx, sy = filter_by_limit(scope[:, 0], scope[:, 1] * 2.5, -0.053, -.003)

x = (x - np.min(x))
sx = (sx - np.min(sx))

shift = 0.4
y = y - np.min(y)
sy = sy - np.min(sy) + shift

peaks, _ = find_peaks(sy, prominence=0.01, distance=100)

xpeaks = x[peaks]
spacing = [xpeaks[i] - xpeaks[i - 1] for i in range(1, len(xpeaks))]
mean_spacing = np.mean(spacing) # Calculate Mhz/ms conversion from mean spacing. This isn't quite accurate due to nonlinearity, but is approximately correct

peaks, _ = find_peaks(y, prominence=0.01, distance=50)
print(x[peaks]*FC/mean_spacing)
x = x * FC / mean_spacing
sx = sx * FC / mean_spacing

# Split points for x-axis break
x1_mask = x < 1000
x2_mask = x > 6000

scope_color = 'white'
sas_color = 'black'

fig, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(13, x6), gridspec_kw={'width_ratios': [3, 3]})

# First subplot (x < 1000)
ax1.plot(x[x1_mask], y[x1_mask], label="Data", color=sas_color)
ax1.set_xlim([np.min(x[x1_mask]), np.max(x[x1_mask])])

# Second subplot (x > 6000)
off = .06
ax2.plot(x[x2_mask], y[x2_mask]-off, label="Data", color=sas_color)
ax2.set_xlim([np.min(x[x2_mask]), np.max(x[x2_mask])])

C = ["#1F77B4", "#FF7F0E", "#2CA02C", "#D62728", "#9467BD", "#8C564B"]
fs = 18
ax1.annotate(
    r'$2 \to 1 (a)$',
    xy=(279, .13),  # Point to annotate
    xytext=(210, .35), # Text position
    arrowprops=dict(color=C[0], shrink=0.01, width=.01, headwidth=5, headlength=5),
    fontsize=fs,
    color=C[0],weight='bold'

)
ax1.annotate(
    r'$2 \to 2 (b)$',
    xy=(439, .13),  # Point to annotate
    xytext=(360, .38), # Text position
    arrowprops=dict(color=C[1], shrink=0.01, width=.01, headwidth=5, headlength=5),
    fontsize=fs,
    color=C[1],weight='bold'

)
ax1.annotate(
    r'$2 \to 3 (c)$',
    xy=(706, .1),  # Point to annotate
    xytext=(706,.35), # Text position
    arrowprops=dict(color=C[2], shrink=0.01, width=.01, headwidth=5, headlength=5),
    fontsize=fs,
    color=C[2],weight='bold'

)
ax2.annotate(
    r'$1 \to 0 (d)$',
    xy=(6914, .09),  # Point to annotate
    xytext=(6914, .40), # Text position
    arrowprops=dict(color=C[3], shrink=0.01, width=.01, headwidth=5, headlength=5),
    fontsize=fs,
    color=C[3],weight='bold'

)
ax2.annotate(
    r'$1 \to 1 (e)$',
    xy=(6988, .1),  # Point to annotate
    xytext=(6988, .37), # Text position
    arrowprops=dict(color=C[4], shrink=0.01, width=.01, headwidth=5, headlength=5),
    fontsize=fs,
    color=C[4],weight='bold'
)
ax2.annotate(
    r'$1 \to 2 (f)$',
    xy=(7138, .12),  # Point to annotate
    xytext=(7040, .32), # Text position
    arrowprops=dict(color=C[5], shrink=0.01, width=.01, headwidth=5, headlength=5),
    fontsize=fs,
    color=C[5],weight='bold'
)

# Break marks
ax1.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax1.yaxis.tick_left()
ax2.yaxis.tick_right()
ax2.yaxis.set_label_position("right")

# Add slanted break markers
d = .010  # Size of diagonal lines
kwargs = dict(transform=ax1.transAxes, color='k', clip_on=False)
ax1.plot((1 - d, 1 + d), (-d, +d), **kwargs)
ax1.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)

kwargs.update(transform=ax2.transAxes)
ax2.plot((-d, +d), (-d, +d), **kwargs)
ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)

ax1.set_xlabel('Detuning (MHz)', fontsize=16)
ax1.set_ylabel('Signal')
ax2.get_yaxis().set_visible(False)
ax1.get_yaxis().set_visible(False)
ax1.set_xticklabels(ax1.get_xticklabels(), fontsize=16)
ax2.set_xticklabels(ax2.get_xticklabels(), fontsize=16)
plt.tight_layout()
plt.show()

fig.savefig('FreqSpectrum.png')

In [ ]:
## Analysis of results of HyperfinePeakGUI.py

import numpy as np
import matplotlib.pyplot as plt

def uncertainty(d): # Uncertainty Propagaion
    L = .539 # Cavity Length
    dL = .001 # Cavity uncertainty
    c = 299792458
    fsr = c/(2*L)
    m = np.mean(d)
    return m*(c/(2*L**2))*dL*1e-6

L = .539 # Cavity Length
dL = .001 # Cavity uncertainty
c = 299792458
FC = c*(1.000293) / (2*L)
arr = np.loadtxt(f"data.csv", # Load output from HyperfinePeakGUI
             delimiter=",", dtype=float)

s1 = uncertainty(arr[:, 0])
arr[:, 0] = arr[:, 0]*FC*1e-6

s2 = uncertainty(arr[:, 1])
arr[:, 1] = arr[:, 1]*FC*1e-6

s3 = uncertainty(arr[:, 2])
arr[:, 2] = arr[:, 2]*FC*1e-6

s4 = uncertainty(arr[:, 3])
arr[:, 3] = arr[:, 3]*FC*1e-6

s5 = uncertainty(arr[:, 4])
arr[:, 4] = arr[:, 4]*FC*1e-6

s6 = uncertainty(arr[:, 5])
arr[:, 5] = arr[:, 5]*FC*1e-6

N = np.sqrt(np.size(arr[:, 0]))

print(f'B->C: {np.mean(arr[:, 0])} +- {np.std(arr[:, 0])} +- {s1}, % Error: {100*(np.abs(np.mean(arr[:, 0]) - 267.1))/267.1 }')
print(f'A->B: {np.mean(arr[:, 1])} +- {np.std(arr[:, 1])} +- {s2}, % Error: {100*(np.abs(np.mean(arr[:, 1]) - 157.2))/157.2}')
print(f'E->F: {np.mean(arr[:, 2])} +- {np.std(arr[:, 2])}+- {s3}, % Error: {100*(np.abs(np.mean(arr[:, 2]) - 157.2))/157.2}')
print(f'F->C: {np.mean(arr[:, 3])} +- {np.std(arr[:, 3])}+- {s4}, % Error: {100*(np.abs(np.mean(arr[:, 3]) - 6834.7))/6834.7}')
print(f'A->E: {np.mean(arr[:, 4])} +- {np.std(arr[:, 4])} +- {s5}, % Error: {100*(np.abs(np.mean(arr[:, 4]) - 6834.7))/6834.7}')
print(f'D->E: {np.mean(arr[:, 5])} +- {np.std(arr[:, 5])/N}+- {s6}, % Error: {100*(np.abs(np.mean(arr[:, 5]) - 72.218))/72.218}')

fig, ax = plt.subplots(1, 6, figsize=(25, 5))
true_values = [267.1, 157.2, 157.2, 6834.7, 6834.7, 72.218]
labels = ['B->', 'A->B', 'E->F', 'F->C', 'A->E', 'D->E']
for i in range(6):
    ax[i].hist(arr[:, i], bins=20)
    ax[i].set_title(labels[i])
    ax[i].axvline(true_values[i], color='r', linestyle='--')
    ax[i].set_xlabel('Mhz')


In [ ]:
## Frequency conversion factor over each frequency sweep

def filter_by_limit(X, Y, min , max):
    X1 = [i for i in X if min <= i <= max]
    Y1 = [Y[i] for i, x in enumerate(X) if min <= x <= max]
    return np.array(X1), np.array(Y1)

fig, ax = plt.subplots()
for k in range(1, 100):
    x, y = filter_by_limit(scope[:, 0]-np.min(scope[:, 0]), scope[:, k], 0.02, 0.08)
    y = savgol_filter(y, polyorder=2, window_length=100)

    peaks, _ = find_peaks(y, prominence=0.01, distance = 20) # Find scope peaks
    peakx = x[peaks]
    spacing = [peakx[i] - peakx[i - 1] for i in range(1, len(peakx))] # Find time spacing between peaks
    x_mean = []
    for i in range(np.size(peakx)-1):
        x_mean.append(peakx[i] + spacing[i]/2) # X value is right in between each individual interferometer cycle
    ax.plot(np.array(x_mean)*1000, FC/(np.array(spacing)*1000), alpha = .3) # Convert to Mhz/Ms -- FC = Mhz/Cycle, so  (Mhz/Cycle) / (ms/cycle) = Mhz/ms
ax.set_xlabel("Time (ms)")
ax.set_ylabel("Mhz/ms")

fig.savefig('peak_spacing.png')

## Zeeman Splitting

In [ ]:
## Load Zeeman Data
data = []
scope = []
arr = np.loadtxt(f"tek0000ALL.csv",
                 delimiter=",", dtype=str)

data.append(arr[2:, 0].astype(float)) # Append common time
scope.append(arr[2:, 0].astype(float))
for i in range(0, 57):
    if i != 15: # Data 15 was corrupted
        arr = np.loadtxt(f"tek{i:04}ALL.csv",
                     delimiter=",", dtype=str)
        data.append(arr[2:, 1].astype(float))
        scope.append(arr[2:, 2].astype(float))
data = np.array(data).T
scope = np.array(scope).T

In [ ]:
# For Zeeman analysis, the calibration is taken from the spacing between the  P3/2 F = 1 to F = 2 hyperfine transitions. The time between the first and last peak in this trace corresponds to 156.9477 Mhz

fig ,ax = plt.subplots()

fy = savgol_filter(data[:, 1], 100, 2)
peaks, _ = find_peaks(fy, prominence=0.001, distance = 10)

for p in peaks:
    ax.axvline(data[:,0][p], color='r')
ax.plot(data[:, 0], fy)
print(data[:, 0][peaks])


In [ ]:
## Plot of various Zeeman Splittings at different magnetic fields

def filter_by_limit(Y, min , max):
    Y1 = [Y[i] for i, x in enumerate(data[:, 0]) if min <= x <= max]
    return np.array(Y1)

# Defne x-axis in MHZ relative to unshifted peak location
maxx = .415; minn = .413
x = (np.array([i for i in data[:, 0] if minn <= i <= maxx])-.4137853)*156.94/(.4137853 - .4113364)

# Prepare data in dictionary format for joyplot
data_dict = {'0 g' : savgol_filter(filter_by_limit(data[:, 1], minn, maxx), 500, 2),
             '10 g' : savgol_filter(filter_by_limit(data[:, 2], minn, maxx), 500, 2),
             '20 g' : savgol_filter(filter_by_limit(data[:, 17], minn, maxx), 500, 2),
             '30 g' : savgol_filter(filter_by_limit(data[:, 32], minn, maxx), 500, 2),
             '52 g' : savgol_filter(filter_by_limit(data[:, 42], minn, maxx), 500, 2),
             '72 g' : savgol_filter(filter_by_limit(data[:, 48], minn, maxx), 500, 2),
             '90 g' : savgol_filter(filter_by_limit(data[:, 56], minn, maxx), 500, 2)}

fig, ax = plt.subplots()
shift = .008
ax.plot(x, data_dict['0 g'], label='0 g')
ax.plot(x, data_dict['10 g']+(shift-.002), label='10 g')
ax.plot(x, data_dict['20 g']+2*(shift-.001), label='10 g')
ax.plot(x, data_dict['30 g']+3*(shift-.0005), label='10 g')

ax.text(x = 70, y =data_dict['0 g'][-1]+.0005, s= '0 G')
ax.text(x = 70, y =data_dict['10 g'][-1]+(shift-.0018), s= '10 G')
ax.text(x = 70, y =data_dict['20 g'][-1]+2*(shift-.0008), s= '20 G')
ax.text(x = 70, y =data_dict['30 g'][-1]+3*(shift-.0002), s= '30 G')
ax.get_yaxis().set_visible(False)
ax.grid(False)
ax.set_xlabel("Detuning (Mhz)")